### Get the packages

In [ ]:
using Pkg; Pkg.instantiate()

# Geodata in Julia

The geodata ecosystem in Julia has matured a lot, but is not in a fully stable state yet.

A brief overview of a problematic situation:
- [YAXArrays.jl](https://github.com/JuliaDataCubes/YAXArrays.jl) named multidimensional arrays
- [Raster.jl](https://github.com/rafaqz/Rasters.jl) for raster data (geotiff, Netcdf, ascii-grid, etc)
- [DimensionalData.jl](https://github.com/rafaqz/DimensionalData.jl) shared backend for Rasters.jl and YAXArrays.jl
- [Shapefile.jl](https://github.com/JuliaGeo/Shapefile.jl) for, you guessed, shapefiles
- [ArchGDAL.jl](https://github.com/yeesian/ArchGDAL.jl) for interactions with the GDAL lib
- [Proj4.jl](https://github.com/JuliaGeo/Proj.jl) for map projections


- https://juliageo.org/ -- biggest geo-group
- https://github.com/JuliaEarth -- for geostatistics
- https://ecojulia.org/ -- (spatial)ecology
- https://github.com/GenericMappingTools/GMT.jl (for Huw)

## Named multidimensional data

This tutorial has been adapted from various sources such as:
- YAXArrays docs
- https://github.com/JuliaDataCubes/datacubes_in_julia_workshop

Let's start to get aquainted with YAXArrays:

In [ ]:
using YAXArrays

yaxa_rand = YAXArray(rand(5, 10))

So far nothing magical has happened, why do we need a whole new package?
If we want to create a `YAXArray` with named dimension we need the following:
- axes or dimensions with names and tick values

In [ ]:
using DimensionalData

axlist = (
    Dim{:time}(range(1, 20, length=20)),
    X(range(1, 10, length=10)),
    Y(range(1, 5, length=15)),
    Dim{:variable}(["temperature", "precipitation"])
)

- the data to feed to the `YAXArray` matching the dimensions defined in the axlist:

In [ ]:
data = rand(20, 10, 15, 2)

- and additionally some metadata:

In [ ]:
props = Dict(
    "origin" => "YAXArrays.jl example",
    "x" => "longitude",
    "y" => "latitude",
);

a2 = YAXArray(axlist, data, props)

We can now access our `YAXArray` at any variable or time point we want:

In [ ]:
a2[variable=At("temperature"), time=1].data

### Reading data

In order to read specific data types (Zarr, NetCDF, etc...) we need to load the necessary backend first:

In [ ]:
using Zarr

bucket = "esdl-esdc-v3.0.2"
store = "esdc-16d-2.5deg-46x72x1440-3.0.2.zarr"
path = "https://s3.bgc-jena.mpg.de:9000/" * bucket * "/" * store
c = Cube(open_dataset(zopen(path,consolidated=true,fill_as_missing=true)))

### Operations on data

Like normal arrays you can modify the data performing simple arithmetics. Additional operations can be performed in different levels of complexity:
- using `map`: applies functions to each element of an array
- using `mapslices`: reduce dimensions
- using `mapCube`: applies functions to an array that may change dimensions

In [ ]:
yaxa_rand.data

In [ ]:
add_yaxa = yaxa_rand .+ 5
add_yaxa.data

Let's apply more complex functions with `map` to each element of the array individually:

In [ ]:
offset = 5
map(a2) do x
    (x + offset) / 2 * 3
end

This computation happens lazily, allowing operations to run fast.

Now let's see how we can apply external functions to the data:

In [ ]:
import Statistics: mean
a2_timemean = mapslices(mean, a2, dims="Time")

We computed the average value of the points in time, so no time variable is present in the final cube.
Similartly we can compute the spatial means in one value per time step:

In [ ]:
a2_spacemean = mapslices(mean, a2, dims=("X", "Y"))

Now to the most flexible way to apply a function: `mapCube`. With it you can directly modify dimension, adding or removing them

Let's use the esdc and apply the median function:

In [ ]:
using Statistics
indims = InDims("time")
outdims = OutDims()
function apply_median(xout, xin)
    x = filter(!ismissing, xin)
    x = filter(!isnan,x)

    xout[] = isempty(x) ? missing : median(x)
end

See how the user defined function passed to mapCube has to have the signature f(outputs..., inputs...) and potentially followd by additional arguments and keyword args.

In [ ]:
medians = mapCube(apply_median, c[Variable=Where(contains("temp"))];indims, outdims)

# Exercise

- Load "bands.zarr"
- Define some spectral indices calulations for your favourite indices. Keep in mind that we have the following bands:
  - B02 (Band 2): Blue (approximately 490 nm)
  - B03 (Band 3): Green (approximately 560 nm)
  - B04 (Band 4): Red (approximately 665 nm)
  - B08 (Band 8): Near Infrared (NIR) (approximately 842 nm)
The results are given leveraging the NDVI = (N-R)/(N+R)
- If you use NDVI you have to divide the data by 10000 to get it in the expected range.
- Apply this calculations to the data in the cube. Use a naive approach, `map` and `mapCube`. Are the results equivalent?

# Exercise solution

In [ ]:
path=pwd()*"/bands.zarr"
bands_cube = Cube(path)

naive computation

In [ ]:
N = bands_cube[bands=At("B08")]
R = bands_cube[bands=At("B04")]
ndvi_naive = (N.-R)./(N.+R)

In [ ]:
#map computation
ndvi(n, r) = (n-r)/(n+r)
ndvi_map = map(ndvi, N, R)

ndvi_naive == ndvi_map

In [ ]:
#mapCube computation

#remodel the function to feed it to mapCube
function ndvi_mc(xout, x1, x2)
    xout .= ndvi.(x1, x2)
end

#provide in and out dimensions
in_dims = InDims("x") # the second one will be inferred
out_dims = OutDims("x") # ditto

put it all together

In [ ]:
ndvi_mapcube = mapCube(ndvi_mc, (N, R), indims=(in_dims, in_dims),
    outdims=OutDims("x", outtype=Float64))

ndvi_mapcube == ndvi_map